In [165]:
import pandas as pd
employees=pd.read_excel("202408-employees.xlsx", dtype={'Emp ID': "str"})


#Charge la liste des employées
print(employees.columns)


employees["mail"]=employees["Email ID"]+"@infosys.com"
employees["mail"]=employees["mail"].str.lower()
# Construction de la colonne nom simple qui contient une concaténation du prénom et du nom en minuscule

employees["Status"]=employees["Status"].str.lower()

# Attention a cette valeur
employeestosuppressaccordingtohr=employees[employees["Status"].str.lower().str.contains('remove')]

employees=employees[~employees["Status"].str.lower().str.contains('remove')]
#employees=employees[employees["Status"]!='Please remove']

employees["nomsimple"]= employees["Prénom"]+" "+employees["Nom"]
employees["nomsimple"]=employees["nomsimple"].str.lower().str.replace("é","e").replace("è","e").replace("ì","i")
employees["nomsimple"]=employees["nomsimple"].fillna("")

employees


Index(['Emp ID', 'Nom', 'Prénom', 'Email ID', 'email', 'location', 'Status'], dtype='object')


,Emp ID,Nom,Prénom,Email ID,email,location,Status,mail,nomsimple
0,15536,MENACHERRY,Antony,Antony_Menacherry,Antony_Menacherry@infosys.com,Marseille,present in france,antony_menacherry@infosys.com,antony menacherry
1,15958,MOHANTY,Sachidananda,Sachidananda_Mohanty,Sachidananda_Mohanty@infosys.com,Marseille,present in france,sachidananda_mohanty@infosys.com,sachidananda mohanty
2,17521,KAMATH,Ajit Venkatesh,Ajit_Kamath,Ajit_Kamath@infosys.com,Paris,present in france,ajit_kamath@infosys.com,ajit venkatesh kamath
3,19406,DHARMAR,Shenbagarajan,Shenbagarajan_d,Shenbagarajan_d@infosys.com,Marseille,present in france,shenbagarajan_d@infosys.com,shenbagarajan dharmar
4,20745,BEIGNON,Pascal,Pascal_Beignon,Pascal_Beignon@infosys.com,Paris,present in france,pascal_beignon@infosys.com,pascal beignon
...,...,...,...,...,...,...,...,...,...
464,1354196,EL HASSOUNI,SOUFIANE,soufiane.hassouni,soufiane.hassouni@infosys.com,Toulouse,please add,soufiane.hassouni@infosys.com,soufiane el hassouni
465,1355598,BOURLIER,C�CILE,cecile.bourlier,cecile.bourlier@infosys.com,Toulouse,please add,cecile.bourlier@infosys.com,c�cile bourlier
466,1356157,COT,OLIVIER CLEMENT ABILIO LOUIS,olivier.cot,olivier.cot@infosys.com,Toulouse,please add,olivier.cot@infosys.com,olivier clement abilio louis cot
467,1153561,Bala Krishna Pemmasani,NaN,balakrishna.p05,balakrishna.p05@infosys.com,Lyon,please add,balakrishna.p05@infosys.com,


In [133]:
#Users est pris d'un extract Comitéo. : Liste des utilisateurs actifs et désactivés (sans adresse)
columnnameemail="Mail de communication"
columnnameemail2="Mail secondaire"
columnnameprenom="Prénom (obligatoire)"
columnnamenom="Nom (obligatoire)"
columnnameidentifiant="Identifiant\n(obligatoire - peut être identique à l’adresse email)"
columnnameetat="Actif (Actif/Inactif) (obligatoire)"

users=pd.read_excel("202408-comiteo.xlsx",dtype={'Matricule': "str"})

users=users.rename(columns={columnnameidentifiant:'Identifiant'})

subset = [columnnameemail, columnnameemail2,"Matricule"]
users.loc[:, subset] = users.loc[:, subset].fillna("")
users=users[users[columnnameetat]=="Actif"]
users

users["mail"]=""

#Construction de la colonne mail qui sera utilisée pour le rapprochement. Elle contiendra le mail qui contient infosys.com
for i, row in users.iterrows():
    if "@infosys.com" in row[columnnameemail]:
        users.loc[i,"mail"]=row[columnnameemail]
    elif "@infosys.com" in  row[columnnameemail2]:
        users.loc[i,"mail"]=row[columnnameemail2]
# Minuscule
users["mail"]=users["mail"].str.lower()
# Construction de la colonne nom simple qui contient une concaténation du prénom et du nom en minuscule
users["nomsimple"]=users[columnnameprenom]+" "+users[columnnamenom]
users["nomsimple"]=users["nomsimple"].str.lower()
users
#users[users.mail.str.contains("arnau")]
#employees[employees.nomsimple.str.contains("malivert")]["Emp ID"]

,Identifiant,Mail de communication,Prénom (obligatoire),Nom (obligatoire),Actif (Actif/Inactif) (obligatoire),Civilité (Mme/M.) (obligatoire),Date de naissance (JJ/MM/AAAA),Mail secondaire,Autorisation des communications CSE (Oui/Non),Autorisation des communications des Bons Plans (Oui/Non),...,Complément d'adresse,Code Postal,Ville,"Pays (Code ISO 2 lettres, ex: France -> FR)",IBAN,BIC,Référence Unique de Mandat (RUM),Date du mandat de prélèvement (JJ/MM/AAAA),mail,nomsimple
0,kagarwal,KrishanKant_A@infosys.com,Krishan Kant,Agarwal,Actif,M.,12/11/1984,krishankant_a@infosys.com,Oui,Oui,...,NaN,31770.0,Colomiers,FR,NaN,NaN,NaN,NaN,krishankant_a@infosys.com,krishan kant agarwal
2,kbangaru_shiva,shivakrishnabangaru@gmail.com,Shiva Krishna,Bangaru,Actif,M.,NaN,shivakrishna_b@infosys.com,Oui,Oui,...,Apt 102,31300.0,Toulouse,FR,NaN,NaN,NaN,NaN,shivakrishna_b@infosys.com,shiva krishna bangaru
3,mbansal1,Mohit_Bansal10@infosys.com,Mohit,Bansal,Actif,M.,NaN,mohit_bansal10@infosys.com,Non,Non,...,Flat 15,31100.0,Toulouse,FR,NaN,NaN,NaN,NaN,mohit_bansal10@infosys.com,mohit bansal
4,mbhaskar_chandra,chandra_baskar@infosys.com,Chandra Mohan,Bhaskar,Actif,M.,NaN,chandu.shine@gmail.com,Oui,Oui,...,APT C12,31300.0,Toulouse,FR,NaN,NaN,NaN,NaN,chandra_baskar@infosys.com,chandra mohan bhaskar
7,vchakravaram_srikanth,Srikanthvarma_Ch@infosys.com,CHAKRAVARAM,Srikanth Varma,Actif,M.,NaN,ch.srikanthvarma@gmail.com,Oui,Oui,...,NaN,31300.0,Apt 72,FR,NaN,NaN,NaN,NaN,srikanthvarma_ch@infosys.com,chakravaram srikanth varma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897,Manikandan_B05,Manikandan_B05@infosys.com,Manikandan,Balakrishnan,Actif,M.,NaN,,Non,Non,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,manikandan_b05@infosys.com,manikandan balakrishnan
898,Pranaykumar.V,Pranaykumar.V@infosys.com,Pranay,Kumar Gupta Vangaveeti,Actif,M.,NaN,,Non,Non,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pranaykumar.v@infosys.com,pranay kumar gupta vangaveeti
899,manasseh.v,manasseh.v@infosys.com,Manasseh,VV,Actif,M.,NaN,,Non,Non,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,manasseh.v@infosys.com,manasseh vv
900,nevin.francis,nevin.francis@infosys.com,Nevin,Francis,Actif,M.,NaN,,Non,Oui,...,NaN,13002.0,Marseille,FR,NaN,NaN,NaN,NaN,nevin.francis@infosys.com,nevin francis


In [113]:
index
#acreer.loc[index,"nomprenom"]=acreer.loc[index,columnnameprenom].fillna("")+" "+acreer.loc[index,columnnamenom]
acreer.loc[index,"nomprenom"].str.lstrip().str.split(" ")
#acreer.loc[index,"nomprenom"].str.split(" ")
prenoms
noms
#acreer.loc[index,columnnamenom]
#acreer.loc[index,columnnamenom]=noms
#acreer
for i in users.columns.to_list():
    if i not in acreer.columns.to_list():
        print (i)
print("Dans a creer pas dans le fichier original\n --------")
for i in acreer.columns.to_list():
    if i not in users.columns.to_list():
        print (i)


Identifiant
mail
nomsimple
Dans a creer pas dans le fichier original
 --------
Identifiant
(obligatoire - peut être identique à l’adresse email)
nomprenom


In [150]:
# Listes des employés à créer
from IPython.core.display import HTML
trouves=[]
nontrouves=[]

#colidentifiant="Identifiant\n(obligatoire - peut être identique à l’adresse email)"
#colnom="Nom (obligatoire)"
#colprenom="Prénom (obligatoire)"
#Premier rapprochement sur l'email, on itère sur les employés
for i,ligne in employees.iterrows():   
    if  len(users.loc[users["mail"]==ligne.mail])>0:
        #print ("Trouvé ")
        indextrouve=users.loc[users["mail"]==ligne.mail].Identifiant.index[0]
        #print( indextrouve)
        trouves.append(indextrouve)
    else:
        #print ("Non trouvé")
        nontrouves.append(i)

#Second rapprochement sur le matricule.
nontrouvesrestant=[]
for i,ligne in employees.loc[nontrouves].iterrows():   
    #print(ligne["Emp No"])
    if  len(users.loc[users["Matricule"]==ligne["Emp ID"]])>0:
        #print ("Trouvé ")
        indextrouve=users.loc[users["Matricule"]==ligne["Emp ID"]].index[0]
        #print( indextrouve)
        trouves.append(indextrouve)
    else:
        #print ("Non trouvé")
        nontrouvesrestant.append(i)

nontrouves=[]
for i,ligne in employees.loc[nontrouvesrestant].iterrows():   
    #print(ligne["Emp No"])
    if  len(users.loc[users["nomsimple"]==ligne["nomsimple"]])>0:
        print ("Trouvé ")
        indextrouve=users.loc[users["nomsimple"]==ligne["nomsimple"]].index[0]
        #print( indextrouve)
        trouves.append(indextrouve)
    else:
        #print ("Non trouvé")
        nontrouves.append(i)

    
employees.loc[nontrouves]  

users.loc[trouves,["Matricule","Identifiant"]].to_excel("users-chequecadeau.xlsx")

display(HTML("<h2>Employés non trouvés dans comitéo</h2>"))

employees.loc[nontrouves]  
acreer=pd.DataFrame(columns=[columnnameidentifiant	,"Mail de communication"	,
                    columnnameprenom,	columnnamenom	,"Actif (Actif/Inactif) (obligatoire)"	
                             ,"Civilité (Mme/M.) (obligatoire)",	
                    "Date de naissance (JJ/MM/AAAA)",	
                             "Mail secondaire",	"Autorisation des communications CSE (Oui/Non)"	,
                    "Autorisation des communications des Bons Plans (Oui/Non)"	,"Situation familiale (Célibataire, Marié(e), Pacsé(e), Divorcé(e), Veuf(ve), Concubinage)"	,
                    "Matricule"	,"Date d'entrée dans l'entreprise (JJ/MM/AAAA)"	,"Date de sortie de l'entreprise (JJ/MM/AAAA)"	,"Site de rattachement"	,
                    "Fonction dans l'entreprise",	                 "Service de l'entreprise",
                    "Type de contrat (Alternant, Apprenti, CDD, CDI, Externe, Retraité(e), Stagiaire)"	,"Téléphone principal (0101010101)"	,
                    "Téléphone secondaire (0101010101)"	,     "Commentaire"	,"Statut"	,"Quotient familial de l'année précédente",
                    "Quotient familial de l'année en cours"	,"Quotient familial de l'année suivante",	"Titre de l'adresse",	"Adresse de facturation par défaut (Oui/Non)",	
                    "Adresse de livraison par défaut (Oui/Non)",	"Adresse",	"Complément d'adresse"	,"Code Postal"	,"Ville",	
                    "Pays (Code ISO 2 lettres, ex: France -> FR)",	 "IBAN",	"BIC",	"Référence Unique de Mandat (RUM)",	"Date du mandat de prélèvement (JJ/MM/AAAA)"])
acreer["Identifiant\n(obligatoire - peut être identique à l’adresse email)"]=employees.loc[nontrouves]['email'].str.replace("@infosys.com","")
acreer["Mail de communication"]=employees.loc[nontrouves]['email']
acreer["Prénom (obligatoire)"]=employees.loc[nontrouves]['Prénom']
acreer["Nom (obligatoire)"]=employees.loc[nontrouves]['Nom']
acreer["Matricule"]=employees.loc[nontrouves]["Emp ID"]
acreer["Civilité (Mme/M.) (obligatoire)"]='M.'
acreer["Actif (Actif/Inactif) (obligatoire)"]='Actif'
acreer["Site de rattachement"]=employees.loc[nontrouves]['location']
acreer["Date d'entrée dans l'entreprise (JJ/MM/AAAA)"]=pd.Timestamp.now().strftime('%d/%m/%Y')
employees.loc[nontrouves]

# Mettre les prénoms aux endroits où ils manquent
#acreer[columnnameprenom].fillna("",inplace=True)
index=acreer[acreer[columnnamenom].isnull()|acreer[columnnameprenom].isna()].index
print(index)

acreer.loc[index,"nomprenom"]=(acreer.loc[index,columnnameprenom].fillna("")+" "+acreer.loc[index,columnnamenom]).str.lstrip()
prenoms=acreer.loc[index,"nomprenom"].str.split(" ").apply(lambda x: x[0])
noms=acreer.loc[index,"nomprenom"].str.split(" ").apply(lambda x: ' '.join(x[1:]))

acreer.loc[index,columnnameprenom]=prenoms
acreer.loc[index,columnnamenom]=noms

print(acreer.loc[index,columnnamenom])
# Ralonger les prenom trop courts
indexnomscourts=acreer[acreer[columnnamenom].apply(lambda x:len(x)<2)].index
acreer.loc[indexnomscourts,columnnamenom]="Name too short "+ acreer.loc[indexnomscourts,columnnamenom]
print(indexnomscourts)
#Suppression de la colonne intermediaire
acreer.drop(columns="nomprenom",inplace=True)


acreer.loc[:,columnnameprenom]=acreer.loc[:,columnnameprenom].str.capitalize()
acreer.loc[:,columnnamenom]=acreer.loc[:,columnnamenom].str.capitalize()
#print(acreer[[colprenom,colnom]])
acreer.to_excel("acreer.xlsx",index=False)
acreer.to_csv("acreer.csv")




Index([467, 468], dtype='int64')
467    Krishna Pemmasani
468                S.G .
Name: Nom (obligatoire), dtype: object
Index([], dtype='int64')


In [ ]:
asupprimer.rename(columns={'Identifiant':'Identifiant\n(obligatoire - peut être identique à l’adresse email)'
                          ,'Nom':'Nom (obligatoire)'
                          ,'Prénom':'Prénom (obligatoire)'
                          ,'Etat du salarié':'Actif (Actif/Inactif) (obligatoire)'
                          ,'Civilité':'Civilité (Mme/M.) (obligatoire)'
                          ,'Mail':'Mail de communication'
                          ,columnnameemail2:'Mail secondaire'
                          ,'Type de contrat':'Type de contrat (Alternant, Apprenti, CDD, CDI, Externe, Retraité(e), Stagiaire)'
                          , 'Site':'Site de rattachement'
                          ,'Fonction':"Fonction dans l'entreprise"
                          ,'Date entrée':"Date d'entrée dans l'entreprise (JJ/MM/AAAA)"
                          , 'Date sortie':"Date de sortie de l'entreprise (JJ/MM/AAAA)"
                          ,'Date de naissance':'Date de naissance (JJ/MM/AAAA)',
                          },inplace='True')
asupprimer[['Adresse',
       'Adresse de facturation par défaut (Oui/Non)',
       'Adresse de livraison par défaut (Oui/Non)',
       'Autorisation des communications CSE (Oui/Non)',
       'Autorisation des communications des Bons Plans (Oui/Non)', 'BIC','Code Postal',
        'Commentaire',"Complément d'adresse", 'Date du mandat de prélèvement (JJ/MM/AAAA)',
        'Pays (Code ISO 2 lettres, ex: France -> FR)',
       "Quotient familial de l'année en cours",
       "Quotient familial de l'année précédente",
       "Quotient familial de l'année suivante",
       'Référence Unique de Mandat (RUM)', "Service de l'entreprise",
       'Situation familiale (Célibataire, Marié(e), Pacsé(e), Divorcé(e), Veuf(ve), Concubinage)',
       "Titre de l'adresse",
        'Statut',
       'Téléphone principal (0101010101)', 'Téléphone secondaire (0101010101)',
       'Ville']]=""


asupprimer.drop(columns=['User ID','Service - Entité - Société','Téléphone 1',
       'Téléphone 2', 'QF N-1', 'QF N', 'QF N+1', 
         'RUM', 'Date du RUM', 'Statut salarié',
       'Date de dernière connexion', 'Date de création', 'mail', 'nomsimple'],inplace=True)

In [151]:

employees.mail[employees.mail=="thibault.fluteaux@infosys.com"]
users.mail[users.mail=="thibaut.fluteaux@infosys.com"]

employees.loc[employees["mail"]=="thibaut.fluteaux@infosys.com"]

,Emp ID,Nom,Prénom,Email ID,email,location,Status,mail,nomsimple


In [181]:
from IPython.core.display import HTML
trouves=[]
nontrouves=[]

#Premier rapprochement sur l'email, on itère sur les utilisateurs comitéos
for i,ligne in users.iterrows():   
    if  len(employees.loc[employees["mail"]==ligne.mail])>0:
        #print ("Trouvé ")
        indextrouve=employees.loc[employees["mail"]==ligne.mail]["Emp ID"].index[0]
        #print( indextrouve)
        trouves.append(indextrouve)
    else:
  #      print ("Non trouvé",ligne.mail)
        nontrouves.append(i)

#Second rapprochement sur le matricule.
nontrouvesrestant=[]
for i,ligne in users.loc[nontrouves].iterrows():   
    if  len(employees.loc[employees["Emp ID"]==ligne["Matricule"]])>0:
        #print ("Trouvé ")
        indextrouve=employees.loc[employees["Emp ID"]==ligne["Matricule"]].index[0]
        #print( indextrouve)
        trouves.append(indextrouve)
    else:
        #print ("Non trouvé")
        nontrouvesrestant.append(i)

nontrouves=[]
for i,ligne in users.loc[nontrouvesrestant].iterrows():   
    #print(ligne["Emp No"])
    if  len(employees.loc[employees["nomsimple"]==ligne["nomsimple"]])>0:
        print ("Trouvé ")
        indextrouve=employees.loc[employees["nomsimple"]==ligne["nomsimple"]].index[0]
        #print( indextrouve)
        trouves.append(indextrouve)
    else:
        #print ("Non trouvé")
        nontrouves.append(i)

#employees.loc[nontrouves]  


display(HTML("<h2>Employés non trouvés dans employees a supprimer</h2>"))

asupprimer=users.loc[nontrouves]
asupprimer[columnnameetat]='Inactif'
asupprimer=asupprimer[~asupprimer[columnnameemail2].str.contains('reception')]
asupprimer.rename(columns={'Identifiant':columnnameidentifiant},inplace=True)

asupprimer["Date de sortie de l'entreprise (JJ/MM/AAAA)"]=pd.Timestamp.now().strftime('%d/%m/%Y')


#print(acreer.columns)
toupdate=pd.concat([asupprimer])
#print(toupdate.columns)

print("Emmployé suprrimé non prévu par les HR")
for i in toupdate.mail:
    if i not in employeestosuppressaccordingtohr.mail.to_list():
        print (i)


toupdate.drop(columns=['nomsimple','mail'],inplace=True)
toupdate['Civilité (Mme/M.) (obligatoire)']=toupdate['Civilité (Mme/M.) (obligatoire)'].fillna("M.")
toupdate.to_csv("tosuppress.csv",index=False)
toupdate.to_excel("tosuppress.xlsx", index=False)

#print(list( set(toupdate.columns)-set(acreer.columns) ))
#print( acreer.columns.sort_values())
#print(toupdate.columns.sort_values())
#print( len(acreer.columns.sort_values()))
#print(len(toupdate.columns.sort_values()))

toupdate.rename(columns={'Identifiant':columnnameidentifiant},inplace=True)

for i in toupdate.columns:
    if i not in users.columns.to_list():
        print(i)
print("------")
for i in users.columns:
    if i not in toupdate.columns.to_list():
        print(i)


Trouvé 


Emmployé suprrimé non prévu par les HR
mohit_bansal10@infosys.com

Identifiant
(obligatoire - peut être identique à l’adresse email)
------
Identifiant
mail
nomsimple
